In [1]:
import numpy as np

In [3]:
# create a list with integers from 1 to 5 called my_list
# multiply each element with the value 3 using either a loop or a list comprehension and save the result under a new variable
my_list = list(range(1, 6, 1))
my_list_times_three = [i * 3 for i in my_list]
print(my_list)
print(my_list_times_three)

[1, 2, 3, 4, 5]
[3, 6, 9, 12, 15]


In [5]:
# import numpy as np and create a numpy array called my_array out of my_list from the cell before
# multiply each element of my_array with the value of 3
my_array = np.array(my_list)
my_array * 3

array([ 3,  6,  9, 12, 15])

In [9]:
# create a numpy array called x with numbers from 1 to 6 using the arange method
# create a 2x3 matrix X by rehaping the array x
# determine the row- and column-wise average of X
x = np.arange(1, 7, 1)
X = x.reshape(2, 3)
print("Vector:")
print(x)
print("Matrix:")
print(X)
print("Columnwise mean:")
print(X.mean(axis = 0))
print("Rowwise mean:")
print(X.mean(axis = 1))

Vector:
[1 2 3 4 5 6]
Matrix:
[[1 2 3]
 [4 5 6]]
Columnwise mean:
[2.5 3.5 4.5]
Rowwise mean:
[2. 5.]


In [11]:
import numpy as np
# create an array y out of the last two elements of x from before
# do this in a way such that y gets not changed once you change the last two elements of x
# examine if this works
x = np.arange(1, 7, 1)
print(f"x before change: {x}")
y = x[-2:].copy()
x[-2:] = [1, 2]
print(f"x after change: {x}")
print(f"y after the change of x: {y}")

x before change: [1 2 3 4 5 6]
x after change: [1 2 3 4 1 2]
y after the change of x: [5 6]


### Portfolio optimization

The weights for a **minimum variance portfolio** for a set of assets can be determined by:

$$
w_{\text{mvp}} = \frac{\Sigma^{-1} \mathbf{1}}{\mathbf{1}^\top \Sigma^{-1} \mathbf{1}}
$$

- $\Sigma$ is the $n \times n$ covariance matrix  
- $\mathbf{1}$ is a vector of ones with dimension $n$

The code below downloads close prices of three stock market listed companies and determines their discrete returns.

1. Estimate the covariance matrix of the returns.
2. Determine its determinant and evaluate if it is different from zero.
3. If the determinant is different from zero determine the inverse matrix of $\Sigma$.
4. Determine the weights of the minimum variance portfolio.
5. Determine the eigenvalues and eigenvectors of $\Sigma$.
6. Determine the trace of the covariance matrix by using the trace method, find out what the trace operations calculates.
7. Is the trace of the covariance matrix equal to the sum of its eigenvalues? If not, examine their values and look into the help of np.isclose, this may be helpful.

In [ ]:
import yfinance as yf

tickers = ["AAPL", "MSFT", "WMT"]
close_prices = yf.download(tickers, start = "2023-01-01", end = "2025-04-30")["Close"]
returns = close_prices.pct_change().dropna().values

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  3 of 3 completed


In [3]:
import numpy as np

cov_hat = np.cov(returns, rowvar=False)
print("Estimated covariance matrix:")
print(cov_hat.round(6))

Estimated covariance matrix:
[[2.76e-04 1.43e-04 6.70e-05]
 [1.43e-04 2.34e-04 4.70e-05]
 [6.70e-05 4.70e-05 1.58e-04]]


In [4]:
print("The determinant of the estimated covariance matrix is different from zero:")
np.linalg.det(cov_hat) != 0

The determinant of the estimated covariance matrix is different from zero:


np.True_

In [18]:
cov_inv = np.linalg.inv(cov_hat)
print("Inverse estimated covariance matrix:")
print(cov_inv.round(6))

Inverse estimated covariance matrix:
[[ 5585.171562 -3122.179967 -1443.938087]
 [-3122.179967  6289.679376  -532.016277]
 [-1443.938087  -532.016277  7099.750499]]


In [ ]:
ones = np.ones(cov_hat.shape[0])
numerator = cov_inv @ ones
denominator = ones @ cov_inv @ ones
weights = numerator / denominator
print(f"The weights of the minimum variance portfolio are: {weights.round(4)}")

The weights of the minimum variance portfolio are: [0.1161 0.3002 0.5837]


In [ ]:
evals, evcs = np.linalg.eig(cov_hat)
print(f"The eigenvalues of the estimated covariance matrix are: {evals.round(6)}")

The eigenvalues of the estimated covariance matrix are: [0.000424 0.000108 0.000136]


In [ ]:
trace = np.trace(cov_hat)
print(f"The trace of the estimated covariance matrix is: {trace.round(6)}")

The trace of the estimated covariance matrix is: 0.000668


In [34]:
print("The trace is equal to the sum of eigenvalues:")
print(trace == evals.sum())
print(np.isclose(trace, evals.sum()))

The trace is equal to the sum of eigenvalues:
False
True


### Solve a linear equation system

Let:

- x = kg of Blend A
- y = kg of Blend B

We now have two equations:
    
1.	Total weight:
$$
x + y = 10
$$
2.	Total cost (price × weight = total price):
$$
10x + 15y = 12 \times 10 = 120
$$
Which gives:
$$
\begin{aligned}
x + y &= 10 \\
10x + 15y &= 120
\end{aligned}
$$

Define the matrix $A$ and the vector $b$ and solve the linear equation system with numpy. Verify the solution by multiplying the matrix $A$ with the solution vector and see if this delivers $b$.

In [36]:
import numpy as np

# Coefficient matrix A
A = np.array([
    [1, 1],        # x + y = 10
    [10, 15]       # 10x + 15y = 120
])

# Right-hand side vector b
b = np.array([10, 120])

# Solve the system
solution = np.linalg.solve(A, b)

print("Blend A (kg):", solution[0])
print("Blend B (kg):", solution[1])

Blend A (kg): 6.0
Blend B (kg): 4.0


In [37]:
A @ solution

array([ 10., 120.])

### Calculate the linear regression OLS estimator

For a linear regression model:

$$
Y = \beta_0 + \beta_1 X_1 + ... + \beta_p X_p + \epsilon
$$

the paramters $\beta_0, ..., \beta_p$ can be estimated by the OLS estimator:

$$
\hat{\mathbf{\beta}} = \left( X^T X \right)^{-1} X^T y
$$

which includes matrix operations and matrix inversion. See the data below and try to calculate $\hat{\mathbf{\beta}}$ for yourself. If you want to, you can do this step by step, i.e.:

1. $X^T X$
2. $\left( X^T X \right)^{-1}$
3. $\left( X^T X \right)^{-1} X^T$
4. $\left( X^T X \right)^{-1} X^T y$

In [40]:
import numpy as np
import pandas as pd

# Set seed for reproducibility
np.random.seed(42)

# Generate 10 observations for two continuous independent variables
x1 = np.random.uniform(0, 10, 10)
x2 = np.random.uniform(0, 5, 10)

# True coefficients
beta_0 = 2.0   # intercept
beta_1 = 1.5   # coefficient for x1
beta_2 = -0.7  # coefficient for x2

# Generate y with some random noise
noise = np.random.normal(0, 1, 10)
y = beta_0 + beta_1 * x1 + beta_2 * x2 + noise

# data 
X = np.column_stack((np.ones(len(x1)), x1, x2))

print("Indepenent variables:")
print(X)
print("Dependent variable:")
print(y)

Indepenent variables:
[[1.         3.74540119 0.10292247]
 [1.         9.50714306 4.84954926]
 [1.         7.31993942 4.1622132 ]
 [1.         5.98658484 1.06169555]
 [1.         1.5601864  0.90912484]
 [1.         1.5599452  0.91702255]
 [1.         0.58083612 1.52121121]
 [1.         8.66176146 2.62378216]
 [1.         6.01115012 2.15972509]
 [1.         7.08072578 1.4561457 ]]
Dependent variable:
[ 6.53322493 13.18027745  9.15833581  8.82438667  5.16954099  3.47222572
  1.87393454 11.73124649  8.96053489 11.71270927]


In [41]:
X.T @ X

array([[ 10.        ,  52.0136736 ,  19.76339204],
       [ 52.0136736 , 360.33620654, 133.06598245],
       [ 19.76339204, 133.06598245,  59.63046475]])

In [42]:
np.linalg.inv(X.T @ X)

array([[ 0.40608752, -0.05067574, -0.02150661],
       [-0.05067574,  0.02209717, -0.03251455],
       [-0.02150661, -0.03251455,  0.09645445]])

In [43]:
np.linalg.inv(X.T @ X) @ X.T

array([[ 0.21407301, -0.17999139, -0.05437095,  0.0798794 ,  0.30747172,
         0.30731409,  0.34393712, -0.08928235,  0.05501965,  0.01594971],
       [ 0.02874056,  0.00172435, -0.02425825,  0.04709031, -0.04575982,
        -0.04602194, -0.0873024 ,  0.05541361,  0.0119312 ,  0.05844236],
       [-0.1333593 ,  0.13713355,  0.14195286, -0.11375244,  0.01545377,
         0.01622338,  0.10633535, -0.05006439, -0.00864133, -0.11128146]])

In [44]:
np.linalg.inv(X.T @ X) @ X.T @ y

array([ 2.1666836 ,  1.28541826, -0.40022204])